In [1]:
import numpy as np
import matplotlib.pylab as plt

# 5. 오차역전파법

앞 장에서는 신경망의 가중치 매개변수의 기울기는 수치 미분을 사용해서 구함  
수치 미분은 시간이 오래 걸린다는 단점  
이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 오차역전파법을 다룸  
오차역전파법을 이해하는 방법 두 가지: 수식, 계산 그래프

## 5.1 계산 그래프

<code>계산 그래프(computational graph)</code>: 계산 과정을 그래프로 나타낸 것, 노드(node)와 에지(edge)로 표현

### 5.1.1 계산 그래프로 풀다

ex. 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.

<img width="601" alt="fig 5-2" src="https://user-images.githubusercontent.com/77653353/194425362-70f6152d-7220-4027-8207-2ae909133086.png">


<code>순전파(forward propagation)</code>: 계산을 왼쪽에서 오른쪽으로 진행하는 단계  
<code>역전파(backward propagation)</code>: 계산을 오른쪽에서 왼쪽으로 반대로 진행하는 단계

### 5.1.2 국소적 계산

계산 그래프는 <code>국소적 계산</code>에 집중함  
전체 계산이 아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 국소적 계산

<img width="643" alt="fig 5-4" src="https://user-images.githubusercontent.com/77653353/194426040-1b161e9b-348a-4ae7-8bc4-77543d484943.png">

위의 그림에서는 여러 식품을 구입(복잡한 계산)을 거쳐 총 금액이 4,000원이 되었는데  
사과와 다른 물품 값을 더하는 계산(4,000 + 200 = 4,200)은  
4,000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 것

### 5.1.3 왜 계산 그래프로 푸는가?

계산 그래프를 사용하는 가장 큰 이유는 역전파를 통해 <code>미분</code>을 효율적으로 계산할 수 있다는 점!

맨 위의 예시에서  
만약 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지를 알고 싶다고 해보자  
이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제에 해당됨  
사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $\frac{\partial{L}}{\partial{x}}$를 구하는 것

<img width="608" alt="fig 5-5" src="https://user-images.githubusercontent.com/77653353/194427222-dbe08b8b-4e0e-4d25-9e89-ea908a9fe499.png">

역전파는 오른쪽에서 왼쪽으로 '1 → 1.1 → 2.2' 순으로 미분 값을 전달함  
사과가 1원 오르면 최종 금액은 2.2원 오른다는 뜻

## 5.2 연쇄법칙

국소적 미분을 전달하는 원리는 <code>연쇄법칙(chain rule)</code>에 따른 것

### 5.2.1 계산 그래프의 역전파

<img width="282" alt="fig 5-6" src="https://user-images.githubusercontent.com/77653353/194427676-a92a8aa2-b16b-474b-83b1-3e375243ab16.png">

역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분 $\frac{\partial{y}}{\partial{x}}$을 곱한 후 다음 노드로 전달하는 것

### 5.2.2 연쇄법칙이란?

연쇄법칙을 설명하려면 우선 합성 함수부터  
<code>합성 함수</code>: 여러 함수로 구성된 함수

ex. $z = (x+y)^2$이라는 식은 아래처럼 두 개의 식으로 구성됨

$$ z = t^2 $$
$$ t = x + y $$

'합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다' 가 연쇄법칙의 원리

수식으로 쓰면

$$ \frac{\partial{z}}{\partial{x}} = \frac{\partial{z}}{\partial{t}} \frac{\partial{t}}{\partial{x}} $$

$$ \frac{\partial{z}}{\partial{t}} = 2t $$  
$$ \frac{\partial{t}}{\partial{x}} = 1 $$  
$$ \frac{\partial{z}}{\partial{x}} = 2t \cdot 1 = 2(x+y) $$

### 5.2.3 연쇄법칙과 계산 그래프

위의 연쇄법칙을 계산 그래프로 나타내면

<img width="466" alt="fig 5-7" src="https://user-images.githubusercontent.com/77653353/194429186-34492d6f-3168-4762-bde2-ff5ace04d458.png">

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

$z = x + y$ 라는 식이 있다면

$$ \frac{\partial{z}}{\partial{x}} = 1 $$  
$$ \frac{\partial{z}}{\partial{y}} = 1 $$

왼쪽이 순전파, 오른쪽이 역전파

<img width="651" alt="fig 5-9" src="https://user-images.githubusercontent.com/77653353/194429956-0449676d-80a9-4e1d-bca8-e19c1661038e.png">

위의 그림에서 상류에서 전해진 미분이 $\frac{\partial{L}}{\partial{z}}$이라고 한다면  
덧셈 노드의 역전파는 <code>입력된 값을 그대로 다음 노드로</code> 보내게 됨

### 5.3.2 곱셈 노드의 역전파

$z = xy$ 라는 식이 있다면

$$ \frac{\partial{z}}{\partial{x}} = y $$  
$$ \frac{\partial{z}}{\partial{y}} = x $$

왼쪽이 순전파, 오른쪽이 역전파

<img width="651" alt="fig 5-12" src="https://user-images.githubusercontent.com/77653353/194430669-ae8cb860-49a4-4916-8e17-c8b3e28ab4b4.png">

위의 그림에서 상류에서 전해진 미분이 $\frac{\partial{L}}{\partial{z}}$이라고 한다면  
곱곱 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 <code>서로 바꾼 값</code>을 곱해서 하류로 보내게 됨

덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값은 필요하지 않지만  
곱셈의 역전파에서는 순방향 입력 신호의 값이 필요하기에 곱셈 노드 구현 시 순전파의 입력 신호를 유지함

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

### 5.4.2 덧셈 계층

In [3]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

$$ y = \begin{cases}
x, & (x>0) \\
0, & (x \le 0)
\end{cases} $$

$$ \frac{\partial{y}}{\partial{x}} = \begin{cases}
1, & (x>0) \\
0, & (x \le 0)
\end{cases} $$

<img width="638" alt="fig 5-18" src="https://user-images.githubusercontent.com/77653353/194433624-d83e7e9a-23a6-472a-ac2e-7c420f9d0fec.png">

In [4]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

여기서 mask라는 인스턴스 변수는 True/False로 구성된 넘파이 배열

In [5]:
x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [6]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


### 5.5.2 Sigmoid 계층

$$ y = \frac{1}{1 + e^{-x}} $$

<img width="644" alt="fig 5-19" src="https://user-images.githubusercontent.com/77653353/194434874-d6dfd4a8-5cf6-4176-acf0-e96d94ceb8e4.png">

<img width="669" alt="fig 5-20" src="https://user-images.githubusercontent.com/77653353/194436706-1af8b171-d15f-4430-9e06-9ad0db8d798f.png">

위의 계산 그래프의 중간 과정을 그룹화하여 아래처럼 단순한 sigmoid 노드 하나로 대체 가능

<img width="305" alt="fig 5-21" src="https://user-images.githubusercontent.com/77653353/194436828-6c066b55-4ef0-4b19-819c-02a06ce11ce0.png">

그리고 $\frac{\partial{L}}{\partial{y}} y^2 e^{-x}$는 아래처럼 정리하여 쓸 수 있음

$$ \begin{matrix}
\frac{\partial{L}}{\partial{y}} y^2 e^{-x} &=& \frac{\partial{L}}{\partial{y}} \frac{1}{(1+e^{-x})^2} e^{-x} \\
&=& \frac{\partial{L}}{\partial{y}} \frac{1}{1+e^{-x}} \frac{e^{-x}}{1+e^{-x}} \\
&=& \frac{\partial{L}}{\partial{y}} y(1-y)
\end{matrix} $$

위의 식에서  

1행: $y^2$ → $\frac{1}{(1+e^{-x})^2}$  
2행: $\frac{1}{(1+e^{-x})^2}$ → $\frac{1}{1+e^{-x}} \frac{e^{-x}}{1+e^{-x}}$  
3행: $\frac{e^{-x}}{1+e^{-x}} = \frac{1+e^{-x}-1}{1+e^{-x}}$ → $(1-y)$

이처럼 sigmoid 계층의 역전파는 순전파의 출력($y$)만으로 계산 가능

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)
        return dx

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층

신경망의 순전파 때 수행하는 <code>행렬의 내적</code>은 기하학에서 <code>어파인 변환(affine transformation)</code>이라고 함  
어파인 변환을 수행하는 처리를 Affine 계층이라는 이름으로 구현

<img width="645" alt="fig 5-25" src="https://user-images.githubusercontent.com/77653353/194439023-65569deb-45ca-4671-bbca-7988c8085f67.png">

행렬의 내적(dot) 노드도 곱셈 노드 처럼 서로 바꾼 값을 하류로 보냄  
다만, 행렬의 형상에 주의

### 5.6.2 배치용 Affine 계층

데이터를 하나만 고려한 것이 아닌 $N$개를 묶어서 순전파, 역전파

<img width="645" alt="fig 5-27" src="https://user-images.githubusercontent.com/77653353/194440287-883db143-a63e-4a7d-8267-ee0203973da2.png">

In [8]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### 5.6.3 Softmax-with-Loss 계층

<img width="569" alt="fig 5-28" src="https://user-images.githubusercontent.com/77653353/194710101-77411fce-ffc4-4f62-9b0a-aa6bc0fc83a8.png">

위의 그림과 같이 Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력함  
학습(Training)시에는 위와 같이 Softmax 계층을 사용하여 예측 결과(output)를 정답(label)과 비교하여 손실(loss)을 구하지만  
추론(Inference)시에는 Softmax 계층을 사용하지 않고, 마지막 Affine 계층의 출력을 인식 결과로 이용함  
신경망에서 정규화하지 않는 출력 결과(위에서는 Softmax 앞의 Affine 계층의 출력)를 <code>점수(score)</code>라고 함  

Softmax-with-Loss 계층의 계산 그래프

<img width="720" alt="fig 5-29" src="https://user-images.githubusercontent.com/78716519/194718957-0dee0a7e-8110-42e3-8119-3b6726bf8994.png">


순전파와 역전파 과정은 "부록 A. Softmax-with-Loss 계층의 계산 그래프" 참고

<img width="643" alt="fig 5-30" src="https://user-images.githubusercontent.com/78716519/194719029-64436a4b-729a-434c-aee8-909f22b29fc2.png">

위의 그림은 '간소화한' Softmax-with-Loss 계층의 계산 그래프  
여기서 주목할 것은 역전파의 결과  
Softmax 계층의 역전파는 $(y_1-t_1,\ y_2-t_2,\ y_3-t_3)$라는 '말끔한' 결과 (왜 이렇게 나오는지는 부록 참고)  
신경망의 역전파에서는 예측결과와 정답의 차이인 오차가 앞 계층에 전해지는 것

분류 문제의 출력층에서는 '소프트맥스 함수'의 손실 함수로 '교차 엔트로피 오차'를 사용하니 역전파가 $(y_1-t_1,\ y_2-t_2,\ y_3-t_3)$로 말끔히 떨어짐  
회귀 문제의 출력층에서는 '항등 함수'의 손실 함수로 '평균 제곱 오차'를 사용해도 역전파가 $(y_1-t_1,\ y_2-t_2,\ y_3-t_3)$로 말끔히 떨어짐  

이런 말끔한 결과는 우연이 아니라 이렇게 설계했기 때문

잘못 예측한 경우  
정답 레이블:         $(0, 1, 0)$  
Softmax 계층의 출력: $(0.3, 0.2, 0.5)$  
정답의 인덱스는 1이지만, 출력에서는 이때의 확률이 겨우 0.2(20%)  
Softmax 계층의 역전파는 $(0.3, -0.8, 0.5)$라는 커다란 오차를 전파

제대로 예측한 경우  
정답 레이블:         $(0, 1, 0)$  
Softmax 계층의 출력: $(0.01, 0.99, 0)$  
정답의 인덱스는 1이고, 출력에서는 이때의 확률이 0.99(99%)  
Softmax 계층의 역전파는 $(0.01, -0.01, 0)$라는 커다란 오차를 전파

In [9]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [10]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

역전파 때는 전파하는 값을 배치의 수(batch_size)로 나눠서 데이터 1개당 오차를 앞 계층으로 전파하는 점에 주의! <span style='color:pink'>!!!질문!!!</span>